## Basics of Flask:

### 1. Create a Flask app that displays "Hello, World!" on the homepage.

In [ ]:
from flask import Flask

app = Flask(__name__)

@app.route('/')
def hello_world():
    return 'Hello, World!'

if __name__ == '__main__':
    app.run(host="0.0.0.0",port=3200)

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:3200
 * Running on http://192.168.1.7:3200
Press CTRL+C to quit


### 2. Build a Flask app with static HTML pages and navigate between them.

In [ ]:
static html pages

home.html:

about.html:

In [ ]:
<!DOCTYPE html>
<html>
<head>
    <title>About</title>
    <link rel="stylesheet" href="{{ url_for('static', filename='style.css') }}">
</head>
<body>
    <h1>About Us</h1>
    <p>This is the about page content.</p>
    <a href="{{ url_for('home') }}">Go to Home Page</a>
</body>
</html>

In [ ]:
from flask import Flask, render_template

app = Flask(__name__)

@app.route('/')
def home():
    return render_template('home.html')

@app.route('/')
def about():
    return render_template('about.html')

if __name__ == '__main__':
    app.run(host="0.0.0.0",port=5264)

### 3. Develop a Flask app that uses URL parameters to display dynamic content.

In [ ]:
from flask import Flask, render_template

app = Flask(__name__)

@app.route('/')
def index():
    return '<h1>Welcome to pwskills lab!</h1>'

@app.route('/user/<username>')
def greet_user(username):
    return f'<h1>Hello, {username}!</h1>'

if __name__ == '__main__':
    app.run(debug=True)

### 4. Create a Flask app with a form that accepts user input and displays it.

In [ ]:
from flask import Flask, render_template, request

app = Flask(__name__)

@app.route('/', methods=['GET', 'POST'])
def index():
    if request.method == 'POST':
        user_input = request.form['user_input']
        return render_template('result.html', user_input=user_input)
    return render_template('form.html')

if __name__ == '__main__':
    app.run(debug=True)

### 5. Implement user sessions in a Flask app to store and display user-specific data.

In [ ]:
from flask import Flask, render_template, request, session, redirect, url_for

app = Flask(__name__)
app.secret_key = 'your_secret_key_here'  # Change this to a secure secret key

@app.route('/', methods=['GET', 'POST'])
def index():
    if request.method == 'POST':
        user_input = request.form.get('user_input')
        session['user_data'] = user_input
        return redirect(url_for('display_data'))
    return render_template('form.html')

@app.route('/display_data')
def display_data():
    user_data = session.get('user_data')
    return render_template('display.html', user_data=user_data)

@app.route('/clear_session')
def clear_session():
    session.pop('user_data', None)
    return redirect(url_for('index'))

if __name__ == '__main__':
    app.run(debug=True)

## Intermediate Flask Topics:

### 6. Build a Flask app that allows users to upload files and display them on the website.

In [ ]:
from flask import Flask, render_template, request, redirect, url_for
import os

app = Flask(__name__)
app.config['UPLOAD_FOLDER'] = 'uploads'

ALLOWED_EXTENSIONS = {'txt', 'pdf', 'png', 'jpg', 'jpeg', 'gif'}

def allowed_file(filename):
    return '.' in filename and filename.rsplit('.', 1)[1].lower() in ALLOWED_EXTENSIONS

@app.route('/')
def index():
    return render_template('upload.html')

@app.route('/upload', methods=['POST'])
def upload_file():
    if 'file' not in request.files:
        return redirect(request.url)

    file = request.files['file']

    if file.filename == '':
        return redirect(request.url)

    if file and allowed_file(file.filename):
        filename = file.filename
        file.save(os.path.join(app.config['UPLOAD_FOLDER'], filename))
        return redirect(url_for('display_file', filename=filename))
    else:
        return "Invalid file format! Allowed formats: txt, pdf, png, jpg, jpeg, gif"

@app.route('/display/<filename>')
def display_file(filename):
    return render_template('display.html', filename=filename)

if __name__ == '__main__':
    app.run(debug=True)

### 7. Integrate a SQLite database with Flask to perform CRUD operations on a list of items.

In [ ]:
from flask import Flask, render_template, request, redirect, url_for
import sqlite3

app = Flask(__name__)

# Function to create a connection to the SQLite database
def create_connection():
    conn = sqlite3.connect('items.db')
    return conn

# Function to create the table if it doesn't exist
def create_table():
    conn = create_connection()
    cursor = conn.cursor()
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS items (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            name TEXT NOT NULL
        )
    ''')
    conn.commit()
    conn.close()

create_table()

@app.route('/')
def index():
    conn = create_connection()
    cursor = conn.cursor()
    cursor.execute('SELECT * FROM items ORDER BY id')
    items = cursor.fetchall()
    conn.close()
    return render_template('index.html', items=items)

@app.route('/add', methods=['POST'])
def add_item():
    if request.method == 'POST':
        item_name = request.form['item_name']
        conn = create_connection()
        cursor = conn.cursor()
        cursor.execute('INSERT INTO items (name) VALUES (?)', (item_name,))
        conn.commit()
        conn.close()
        return redirect(url_for('index'))

@app.route('/delete/<int:item_id>')
def delete_item(item_id):
    conn = create_connection()
    cursor = conn.cursor()
    cursor.execute('DELETE FROM items WHERE id = ?', (item_id,))
    conn.commit()
    conn.close()
    return redirect(url_for('index'))

if __name__ == '__main__':
    app.run(debug=True)

### 8. Implement user authentication and registration in a Flask app using Flask-Login.

In [ ]:
from flask import Flask, render_template, redirect, url_for, request, flash
from flask_login import LoginManager, UserMixin, login_user, logout_user, login_required, current_user
from werkzeug.security import generate_password_hash, check_password_hash

app = Flask(__name__)
app.secret_key = 'your_secret_key_here'  # Change this to a secure secret key

# Flask-Login configuration
login_manager = LoginManager()
login_manager.init_app(app)

# Mock user database (replace this with a database in a real application)
class User(UserMixin):
    def __init__(self, id, username, password):
        self.id = id
        self.username = username
        self.password = password

users = [
    User(1, 'user1', generate_password_hash('password1')),
    User(2, 'user2', generate_password_hash('password2'))
]

# Function to retrieve user by ID
def get_user(user_id):
    for user in users:
        if user.id == int(user_id):
            return user
    return None

@login_manager.user_loader
def load_user(user_id):
    return get_user(user_id)

@app.route('/')
def home():
    return render_template('home.html')

@app.route('/login', methods=['GET', 'POST'])
def login():
    if request.method == 'POST':
        username = request.form['username']
        password = request.form['password']

        user = next((user for user in users if user.username == username), None)
        if user and check_password_hash(user.password, password):
            login_user(user)
            return redirect(url_for('dashboard'))
        else:
            flash('Invalid username or password', 'error')
    
    return render_template('login.html')

@app.route('/logout')
@login_required
def logout():
    logout_user()
    return redirect(url_for('home'))

@app.route('/dashboard')
@login_required
def dashboard():
    return render_template('dashboard.html', username=current_user.username)

if __name__ == '__main__':
    app.run(debug=True)


In [ ]:
home.html:

In [ ]:
login.html:

In [ ]:
dashboard.html:

### 9. Create a RESTful API using Flask to perform CRUD operations on resources like books or movies.

In [ ]:
from flask import Flask, jsonify, request

app = Flask(__name__)

# Mock database for storing book data
books = [
    {'id': 1, 'title': 'Book 1', 'author': 'Author 1'},
    {'id': 2, 'title': 'Book 2', 'author': 'Author 2'},
    {'id': 3, 'title': 'Book 3', 'author': 'Author 3'}
]

# Route to get all books
@app.route('/api/books', methods=['GET'])
def get_books():
    return jsonify({'books': books})

# Route to get a specific book by ID
@app.route('/api/books/<int:book_id>', methods=['GET'])
def get_book(book_id):
    book = next((book for book in books if book['id'] == book_id), None)
    if book:
        return jsonify({'book': book})
    return jsonify({'message': 'Book not found'}), 404

# Route to create a new book
@app.route('/api/books', methods=['POST'])
def create_book():
    new_book = {
        'id': len(books) + 1,
        'title': request.json['title'],
        'author': request.json['author']
    }
    books.append(new_book)
    return jsonify({'message': 'Book created', 'book': new_book}), 201

# Route to update a book by ID
@app.route('/api/books/<int:book_id>', methods=['PUT'])
def update_book(book_id):
    book = next((book for book in books if book['id'] == book_id), None)
    if book:
        book['title'] = request.json.get('title', book['title'])
        book['author'] = request.json.get('author', book['author'])
        return jsonify({'message': 'Book updated', 'book': book})
    return jsonify({'message': 'Book not found'}), 404

# Route to delete a book by ID
@app.route('/api/books/<int:book_id>', methods=['DELETE'])
def delete_book(book_id):
    global books
    books = [book for book in books if book['id'] != book_id]
    return jsonify({'message': 'Book deleted'})

if __name__ == '__main__':
    app.run(debug=True)

### 10. Design a Flask app with proper error handling for 404 and 500 errors.

In [ ]:
from flask import Flask, render_template

app = Flask(__name__)

# Route for handling 404 error (Not Found)
@app.errorhandler(404)
def not_found_error(error):
    return render_template('404.html'), 404

# Route for handling 500 error (Internal Server Error)
@app.errorhandler(500)
def internal_error(error):
    return render_template('500.html'), 500

# Route to deliberately raise a 500 error for testing
@app.route('/simulate_error')
def simulate_error():
    raise Exception('Simulated Internal Server Error')

# Default route
@app.route('/')
def index():
    return render_template('index.html')

if __name__ == '__main__':
    app.run(debug=True)

## Real-time Development:

### 11. Create a real-time chat application using Flask-SocketIO.

In [ ]:
from flask import Flask, render_template
from flask_socketio import SocketIO, emit

app = Flask(__name__)
app.config['SECRET_KEY'] = 'your_secret_key_here'  # Change this to a secure secret key
socketio = SocketIO(app)

@app.route('/')
def index():
    return render_template('index.html')

@socketio.on('message')
def handle_message(data):
    emit('message', data, broadcast=True)

if __name__ == '__main__':
    socketio.run(app, debug=True)


In [ ]:
Create an index.html file in a templates folder:
index.html:

In [ ]:
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <title>Chat Application</title>
    <script src="https://cdnjs.cloudflare.com/ajax/libs/socket.io/4.3.2/socket.io.js"></script>
    <script>
        document.addEventListener('DOMContentLoaded', function() {
            var socket = io.connect('http://' + document.domain + ':' + location.port);

            socket.on('message', function(data) {
                var div = document.createElement('div');
                div.innerHTML = '<b>' + data.username + '</b>: ' + data.message;
                document.getElementById('chat').appendChild(div);
            });

            document.getElementById('send_button').onclick = function() {
                var message = document.getElementById('message').value;
                var username = document.getElementById('username').value;
                socket.emit('message', {'message': message, 'username': username});
                document.getElementById('message').value = '';
            };
        });
    </script>
</head>
<body>
    <div id="chat"></div>
    <input type="text" id="username" placeholder="Enter your username"><br>
    <input type="text" id="message" placeholder="Type a message">
    <button id="send_button">Send</button>
</body>
</html>

### 12. Build a Flask app that updates data in real-time using WebSocket connections.

In [ ]:
from flask import Flask, render_template
from flask_socketio import SocketIO, emit
import random
import time

app = Flask(__name__)
app.config['SECRET_KEY'] = 'your_secret_key_here'  # Change this to a secure secret key
socketio = SocketIO(app)

@app.route('/')
def index():
    return render_template('index.html')

@socketio.on('connect')
def handle_connect():
    print('Client connected')
    # Simulate sending data updates every 2 seconds
    while True:
        random_number = random.randint(1, 100)
        emit('update_data', {'data': random_number}, broadcast=True)
        time.sleep(2)

if __name__ == '__main__':
    socketio.run(app, debug=True)


In [ ]:
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <title>Real-time Data Update</title>
    <script src="https://cdnjs.cloudflare.com/ajax/libs/socket.io/4.3.2/socket.io.js"></script>
    <script>
        document.addEventListener('DOMContentLoaded', function() {
            var socket = io.connect('http://' + document.domain + ':' + location.port);

            socket.on('update_data', function(data) {
                document.getElementById('data').innerText = data.data;
            });
        });
    </script>
</head>
<body>
    <h1>Real-time Data: <span id="data">Waiting for data...</span></h1>
</body>
</html>

### 13. Implement notifications in a Flask app using websockets to notify users of updates.

In [ ]:
from flask import Flask, render_template
from flask_socketio import SocketIO, emit

app = Flask(__name__)
app.config['SECRET_KEY'] = 'your_secret_key_here'  # Change this to a secure secret key
socketio = SocketIO(app)

@app.route('/')
def index():
    return render_template('index.html')

@socketio.on('connect')
def handle_connect():
    print('Client connected')

# Route to trigger notifications (simulated updates)
@app.route('/trigger_notification')
def trigger_notification():
    emit('notification', {'message': 'New update!'}, broadcast=True)
    return 'Notification sent'

if __name__ == '__main__':
    socketio.run(app, debug=True)


In [ ]:
HTML Template:

Create an index.html file in a templates folder:

index.html:

html
Copy code
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <title>Notifications</title>
    <script src="https://cdnjs.cloudflare.com/ajax/libs/socket.io/4.3.2/socket.io.js"></script>
    <script>
        document.addEventListener('DOMContentLoaded', function() {
            var socket = io.connect('http://' + document.domain + ':' + location.port);

            socket.on('notification', function(data) {
                alert(data.message);
            });
        });
    </script>
</head>
<body>
    <h1>Real-time Notifications</h1>
    <button onclick="triggerNotification()">Trigger Notification</button>
    
    <script>
        function triggerNotification() {
            fetch('/trigger_notification')
                .then(response => response.text())
                .then(data => console.log(data))
                .catch(error => console.error('Error:', error));
        }
    </script>
</body>
</html>